## 1.简介
java虚拟机支持多线程执行。一个线程由线程类来表示。
## 2.举例
有共享变量而未进行同步的线程，会产生出乎意料的结果，因为jvm允许编译器和计算机处理器为了优化运行速度而进行指令重排。

## 3.非正式内存语义
正确的同步有利于编程人员正确理解程序的行为，并且使程序更少的依赖可能的重排序。
理解一个程序是否正确同步的两个关键要素：
 1. Conflicting Accesses 冲突访问
 2. Happens-Before Relationship 先序关系：
    如果两个操作存在先序关系，那么先发生的操作，一定对后一个是可见的，并且在排序中，会被排在后一个操作之前。这种关系不是规定所有操作的顺序，而是定义在两个操作发生冲突时，操作执行的顺序。jvm中定义的先序关系有以下几种
    * 线程中每个操作，都发生在代码块中排在他后面的操作之前
    * 一个管程的锁操作，发生在该锁的解锁操作之前
    * volatile变量的写操作，发生在该变量时间上之后的读操作之前（非volatile变量，可能时间上之后的读操作，也无法读到之前写操作的结果）
    * 对线程调用start的操作，发生在线程所有的操作之前
    * 线程中的所有操作，发生在等待该线程的其他线程，获得join成功返回之前
    * 传递性：如果a操作先于b发生，b先于c发生，则a先于c发生。

当一个程序包含冲突访问，这两个冲突访问中又不存在先序关系，这种情况叫做数据竞争。一个正确同步的程序，是不包含数据竞争的。
如果一个程序是正确同步的，那么这个程序会表现为顺序一致的。这是一种对编程人员的强有力保证，不存在数据竞争的程序，也无需担心指令重排的影响。


### 3.1顺序一致性

顺序一致性是一种对可见性，以及程序执行顺序的强保证。在执行一个顺序一致性的程序时，所有动作（如读和写）间存在一个全序关系，与程序的顺序一致。
每个操作都是原子的，并对所有线程可见。如果一个程序不包含数据竞争，那么程序的执行会表现为顺序一致的。
当然，即使在数据一致性的且不存在数据竞争的程序中，如果某个需要保持原子性的操作，而没有得到保障，仍然会出现错误。

如果以顺序一致性作为java内存模型的要求，那么很多java编译器优化以及处理器执行指令优化将是非法的。

讨论完顺序一致性，可以用它为数据争用和未正确同步的程序提供一个重要说明。程序的某次执行中存在冲突访问，且没有被同步排序，就会发生数据竞争。一个程序是正确同步的当且仅当所有顺序一致的执行过程中都不存在数据争用。因此，编程人员只需推断顺序一致的执行过程来确定程序是否正确同步了。
4-6节更加详细地讨论了非final字段的内存模型问题。
### 3.2 final域

* final域被初始化一次，并且永不改变。final域的内存语义规则与普通域有所不同。编译器可以把final域的读操作随意移除同步栅栏，并随意调用其他方法？
可以把final域的值保存在寄存器中，不需要实时刷新。

* final域也有助于编程人员定义无需同步的线程安全对象：不可变对象。

* final域必须被正确使用，才可以保证不不变性。一个完全初始化的对象,是一个构造器执行完毕的对象。一个线程只有在看到一个对象引用之前，这个对象已经完全初始化，才能保证这个线程看到这个对象正确的final域。

* final域的使用方法很简单，将一个final域在构造函数中初始化。不要将一个正在初始化对象的引用写入到一个可以在构造函数完成前，就可以被其他线程看到的地方。遵守以上规范，那么其他线程将总是会看到一个完全构造的对象。他们会看到对象中final域引用的不同版本的对象或者数组，但这些对象都是实时更新的，至少和final字段一样新。
> final字段所引用的对象里的字段或数组元素可能在后续还会变化，若没有正确同步，其它线程也许不能看到最新改变的值，但一定可以看到完全初始化的对象或数组被final字段引用的那个时刻的对象字段值或数组元素


内存模型描述了某个程序的可能行为。JVM实现可以自由地生成想要的代码，只要该程序所有最终执行产生的结果能通过内存模型进行预测。这为大量的代码转换提供了充分的自由，包括动作（action）的重排序以及非必要的同步移除。
内存模型的一个高级、非正式的概述显示其是一组规则，规定了一个线程的写操作何时会对另一个线程可见。通俗地说，读操作r通常能看到任何写操作w写入的值，意味着w不是在r之后发生，且w看起来没有被另一个写操作w'覆盖掉（从r的角度看）。




## 4.什么是内存模型

java 内存模型 定义了给定的一个程序，某程序的执行序列轨迹，是否合法。对于Java，内存模型检查执行轨迹中的每次读操作，然后根据特定规则，检验该读操作观察到的写是否合法。

## 5.定义

### 共享变量/堆内存

### 线程间动作

一个线程间动作是被一个线程执行，同时可以被另外一个线程检测到，或者影响的动作。
线程间动作包括
 * 共享变量的读写
 * 同步动作（如管程的加锁与解锁操作，volatile变量的读写操作，线程启动动作）
 * 线程与外界交互的动作
 * 导致线程进入无线循环的动作（线程分散动作）

我们无需考虑线程内动作，线程内动作只需每个线程各自遵守线程内语义

每个线程间动作都与该动作的执行信息相关。
所有的动作都与导致该动作发生的线程有关，也和他们在该线程内发生的顺序有关。
其他与动作有关的信息如下
 *  写
 * 读
 * 锁
 * 解锁


### 程序顺序
在所有由线程t执行的的线程间动作中，t的程序顺序是一个全序，反映出的是根据t的线程内语义，这些动作的执行顺序。
### 线程内语义

线程内语义是为单线程程序的标准语义，可以完整的预测这个线程的行为。为了确定线程t中的动作在某次执行中是否合法，只需简单的看线程t的实现是否像是在单线程上下文中执行，如同JLS其它部分定义的那样。

每次线程t的执行都生成一个线程间的动作，该动作必须与t中在程序顺序上紧跟着该动作的线程间动作a相配。如果a是一个读操作，那么线程t后续就会使用到a看到的值，该值由内存模型确定。
简而言之，线程内语义决定着某个线程孤立的执行过程；当从堆中读取值时，值是由内存模型决定的。





























